In [2]:
import gym
import sys
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from collections import deque
import matplotlib.pyplot as plt

In [3]:
class A2C(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(A2C, self).__init__()
        self.common = Dense(64, activation='relu')
        self.actor  = Dense(action_size, activation='softmax',
                                        kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))
        self.critic = Dense(1,
                                        kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))
        
    def call(self, x):
        x      = self.common(x)
        policy = self.actor(x)
        value  = self.critic(x)
        return policy, value

In [4]:
class A2CAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size= action_size
        
        # Hyper params for learning
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        
        self.model = A2C(self.state_size,self.action_size)
        self.model.load_weights("./save_model/cartpole_a2c")
        
    def get_action(self, state):
        policy, _ = self.model(state)
        policy = np.array(policy[0])
        return np.random.choice(self.action_size, 1, p=policy)[0]
        
    def get_loss(self, state, action, reward, next_state, done):
        model_params = self.model.trainable_variables
        policy, value      = self.model(state)
        _,      next_value = self.model(next_state)
        target = reward + (1 - done) * self.discount_factor * next_value[0]

        # For policy network
        one_hot_action = tf.one_hot([action], self.action_size)
        action_prob = tf.reduce_sum(one_hot_action * policy, axis=1)
        cross_entropy = - tf.math.log(action_prob + 1e-5)
        advantage = tf.stop_gradient(target - value[0])
        actor_loss = tf.reduce_mean(cross_entropy * advantage)

        # For value network
        critic_loss = 0.5 * tf.square(tf.stop_gradient(target) - value[0])
        critic_loss = tf.reduce_mean(critic_loss)

        # integrate losses
        loss = 0.2 * actor_loss + critic_loss
            
        return np.array(loss)

In [5]:
%matplotlib tk

ENV_NAME = 'CartPole-v1'
EPISODES = 10
# END_SCORE = 300

if __name__ == "__main__":
    env = gym.make(ENV_NAME)
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    agent = A2CAgent(state_size, action_size)
    print('Env Name : ',ENV_NAME)
    print('States {}, Actions {}'
            .format(state_size, action_size))

    scores, episodes, losses = [], [], []
    score_avg = 0
    
    for e in range(EPISODES):
        # Episode initialization
        done = False
        score = 0
        loss_list = []
        
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        
        while not done:
            env.render()

            # Interact with env.
            action = agent.get_action(state)
            next_state, reward, done, info = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])
            loss = agent.get_loss(state, action, reward, next_state, done)
            state = next_state

            # 
            score += reward
            loss_list.append(loss)
            if done:
                print('epi: {:3d} | score {:3.2f} | loss {:.4f}'
                      .format(e+1, score, tf.reduce_mean(loss_list)))
                scores = np.append(scores,score)
                losses.append(tf.reduce_mean(loss_list))
    print('Avg. score {:4.2f}, Avg. loss {:.4f}'.
          format(tf.reduce_mean(scores), tf.reduce_mean(losses)))
    env.close()     

epi:   1 | score 500.00 | loss 0.5657
epi:   2 | score 500.00 | loss 0.5672
epi:   3 | score 500.00 | loss 0.5719
epi:   4 | score 500.00 | loss 0.5730
epi:   5 | score 500.00 | loss 0.5690
epi:   6 | score 500.00 | loss 0.5655
epi:   7 | score 500.00 | loss 0.5718
epi:   8 | score 500.00 | loss 0.5705
epi:   9 | score 500.00 | loss 0.5607
epi:  10 | score 500.00 | loss 0.5713
Avg. score 500.00, Avg. loss 0.5687


C:\Users\user\.conda\envs\tf240\lib\site-packages\ipykernel\eventloops.py:256: RuntimeWarning: coroutine 'Kernel.do_one_iteration' was never awaited
  self.func()
